In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr 29 17:15:44 2021

@author: anishayallapragada
"""

from Bio import SeqIO
from gensim.models import word2vec

import numpy as np
import sys
import os
import gzip

"""
 'AGAMQSASM' => [['AGA', 'MQS', 'ASM'], ['GAM','QSA'], ['AMQ', 'SAS']]
"""
def split_ngrams(seq, n):
    a, b, c = zip(*[iter(seq)]*n), zip(*[iter(seq[1:])]*n), zip(*[iter(seq[2:])]*n)
    str_ngrams = []
    for ngrams in [a,b,c]:
        x = []
        for ngram in ngrams:
            x.append("".join(ngram))
        str_ngrams.append(x)
    return str_ngrams


'''
Args:
    corpus_fname: corpus file name
    n: the number of chunks to split. In other words, "n" for "n-gram"
    out: output corpus file path
Description:
    Protvec uses word2vec inside, and it requires to load corpus file
    to generate corpus.
'''
def generate_corpusfile(corpus_fname, n, out):
    f = open(out, "w")
    with gzip.open(corpus_fname, 'rb') as fasta_file:
        for r in SeqIO.parse(fasta_file, "fasta"):
            ngram_patterns = split_ngrams(r.seq, n)
            for ngram_pattern in ngram_patterns:
                f.write(" ".join(ngram_pattern) + "\n")
                sys.stdout.write(".")

    f.close()


def load_protvec(model_fname):
    return word2vec.Word2Vec.load(model_fname)

def normalize(x):
    return x / np.sqrt(np.dot(x, x))

class ProtVec(word2vec.Word2Vec):

    """
    Either fname or corpus is required.
	corpus_fname: fasta file for corpus
    corpus: corpus object implemented by gensim
    n: n of n-gram
    out: corpus output file path
    min_count: least appearance count in corpus. if the n-gram appear k times which is below min_count, the model does not remember the n-gram
    """
    def _init_(self, corpus_fname=None, corpus=None, n=3, size=100,
                   sg=1, window=25, min_count=1, workers=3):
        skip_gram = True 
        
        self.n = n
        self.size = size
        self.corpus_fname = corpus_fname
        self.sg = int(skip_gram)
        self.window = window
        self.min_count = min_count
        self.workers = workers
        out = corpus.txt

        directory = out.split('/')[0]
        if not os.path.exists(directory):
            os.makedirs(directory)
            print("directory(trained_models) created\n")

        if corpus is None and corpus_fname is None:
            raise Exception("Either corpus_fname or corpus is needed!")

        if corpus_fname is not None:
            print ('Now we are checking whether corpus file exist')
            if not os.path.isfile(out):
                print ('INFORM : There is no corpus file. Generate Corpus file from fasta file...')
                generate_corpusfile(corpus_fname, n, out)
            else:
                print( "INFORM : File's Existence is confirmed")
            self.corpus = word2vec.Text8Corpus(out)
            print ("\n... OK\n")

    def word2vec_init_(self, ngram_model_fname):
        word2vec.Word2Vec._init_(self, self.corpus, size=self.size, sg=self.sg, window=self.window, min_count=self.min_count, workers=self.workers)
        model = word2vec.Word2Vec([line.rstrip().split() for line in open(self.out)], min_count = 1, size=self.size, sg=self.sg, window=self.window)
        model.wv.save_word2vec_format(ngram_model_fname)
    
    def to_vecs(self, seq, ngram_vectors):
        ngrams_seq = split_ngrams(seq, self.n)


        protvec = np.zeros(self.size, dtype=np.float32)
        for index in xrange(len(seq) + 1 - self.n):
            ngram = seq[index:index + self.n]
            if ngram in ngram_vectors:
                ngram_vector = ngram_vectors[ngram]
                protvec += ngram_vector
        return normalize(protvec)
        
    def get_ngram_vectors(self, file_path):
        ngram_vectors = {}`
        vector_length = None
        with open(file_path) as infile:
            for line in infile:
                line_parts = line.rstrip().split()   
                # skip first line with metadata in word2vec text file format
                if len(line_parts) > 2:     
                    ngram, vector_values = line_parts[0], line_parts[1:]          
                    ngram_vectors[ngram] = np.array(map(float, vector_values), dtype=np.float32)
        return ngram_vectors

/Users/anishayallapragada/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [7]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
from Bio import SeqIO


from collections import Counter
from gensim.models import Word2Vec


import numpy as np
import os
import sys
import gzip


def make_protein_vector_for_uniprot(fasta_file, protein_vector_fname, ngram_vectors):
    with gzip.open(fasta_file, 'rb') as fasta_file:
        with open(protein_vector_fname, 'w') as output_file:
            for record in SeqIO.parse(fasta_file, "fasta"):
                protein_name = record.name.split('|')[-1]
                protein_vector = pv.to_vecs(record.seq, ngram_vectors)

                output_file.write('{}\t{}\n'.format(protein_name, ' '.join(map(str, protein_vector))))

def make_protein_vector_for_other(fasta_file, protein_vector_fname, ngram_vectors):
    with gzip.open(fasta_file, 'rb') as fasta_file:
        with open(protein_vector_fname, 'w') as output_file:
            for record in SeqIO.parse(fasta_file, "fasta"):
                protein_name = record.name.split(' ')[-1]
                protein_vector = pv.to_vecs(record.seq, ngram_vectors)

                output_file.write('{}\t{}\n'.format(protein_name, ' '.join(map(str, protein_vector))))

def get_uniprot_protein_families(path):
    protein_families = {}
    protein_family_stat = Counter()
    for record in SeqIO.parse(path, "fasta"): 
        family_id = None
        for element in record.description.split():
            if element.startswith('PFAM'):
                family_id = element.split('=', 1)[1]
        if family_id:
            uniprot_id = record.name.split('|')[-1]
            protein_families[uniprot_id] = family_id
            protein_family_stat[family_id] += 1

    return protein_families, protein_family_stat

def make_uniport_with_families(Pfam_file, fasta_file, uniprot_with_families): 
    protein_families = {}
    protein_family_stat = Counter()
    with gzip.open(Pfam_file, 'rb') as gzipped_file:
        for record in SeqIO.parse(gzipped_file, "fasta"):  
            family_id = record.description.rsplit(';', 2)[-2]
            uniprot_id = record.name.split('/', 1)[0].lstrip('>') 
            protein_families[uniprot_id] = family_id

    with gzip.open(fasta_file, 'rb') as gzipped_file, open(uniprot_with_families, "w") as output_fasta:
        for record in SeqIO.parse(gzipped_file, "fasta"):
            uniprot_id = record.name.split('|')[2] 
            if uniprot_id in protein_families:
                family = protein_families[uniprot_id]
                record.description += ' PFAM={}'.format(protein_families[uniprot_id])
                SeqIO.write(record, output_fasta, "fasta")

def make_protein_pfam_vector_for_uniprot(protein_pfam_vector_fname, protein_vector_fname, protein_families, protein_family_stat):
    #Cut standard
    min_proteins_in_family = 100

    f = open(protein_pfam_vector_fname, "w")
    with open(protein_vector_fname) as protein_vector_file:
        for line in protein_vector_file:
            uniprot_name, vector_string = line.rstrip().split('\t', 1)
            if uniprot_name in protein_families:
                family = protein_families[uniprot_name]
                if protein_family_stat[family] >= min_proteins_in_family:
                    f.write('{},{},{}'.format(uniprot_name, protein_families[uniprot_name], vector_string) + "\n")
    f.close()


def make_protein_pfam_vector_for_other(protein_pfam_vector_fname, protein_vector_fname, fasta_file):
    #Cut standard
    min_proteins_in_family = 0

    protein_families = {}
    f = open(protein_pfam_vector_fname, "w")
    with open(protein_vector_fname) as protein_vector_file, gzip.open(fasta_file, 'rb') as gzipped_fasta:
        for record in SeqIO.parse(gzipped_fasta, "fasta"):
            gz_protein_name, gz_family = record.description.rstrip().split(' ', 1)
            print (gz_protein_name)
            print (gz_family)
            protein_families[gz_protein_name] = gz_family
        
        for line in protein_vector_file:
            protein_name, vector_string = line.rstrip().split('\t', 1)
            if protein_name in protein_families:
                family = protein_families[protein_name]
                f.write('{}\t{}\t{}'.format(protein_name, protein_families[protein_name], vector_string) + "\n")
                
    f.close()

fasta_file = "document/uniprot.fasta.gz"
Pfam_file = "document/Pfam-A.fasta.gz"
ngram_corpus_fname = "trained_models/ngram_vector.csv"
model_ngram = "trained_models/ngram_model"
protein_vector_fname = "trained_models/protein_vector.csv"
uniprot_with_families = "trained_models/uniprot_with_families.fasta"
protein_pfam_vector_fname = "trained_models/protein_pfam_vector.csv"

#Make corpus

pv = Word2Vec.ProtVec(fasta_file)

print ("Checking the file(trained_models/ngram_vector.csv)")
if not os.path.isfile(ngram_corpus_fname) or not os.path.isfile(protein_vector_fname):
    print ('INFORM : There is no vector model file. Generate model files from data file...')
    
    #Make ngram_vector.txt and word2vec model
    pv.word2vec_init_(ngram_corpus_fname)
    pv.save(model_ngram) 

    #Get ngram and vectors
    ngram_vectors = pv.get_ngram_vectors(ngram_corpus_fname)
    
    #Make protein_vector.txt by ngram, vector, uniprot
    make_protein_vector_for_uniprot(fasta_file, protein_vector_fname, ngram_vectors)

else:
    print("INFORM : File's Existence is confirmed\n")

print ("...OK\n")

print("Checking the file(trained_models/protein_pfam_vector.csv)")
if not os.path.isfile(protein_pfam_vector_fname):
    print ('INFORM : There is no pfam_model file. Generate pfam_model files from data file...')
    
    #Make uniprot_with_family.fasta by uniprot, Pfam
    make_uniport_with_families(Pfam_file, fasta_file, uniprot_with_families)

    #Get protein_name, family_name, vectors
    protein_families, protein_family_stat = get_uniprot_protein_families(uniprot_with_families)

    #Make protein_pfam_vector_fname.csv by protein_name, family_name, vectors
    make_protein_pfam_vector_for_uniprot(protein_pfam_vector_fname, protein_vector_fname, protein_families, protein_family_stat)

print ("...Uniprot Done\n")